# Search

**Table of contents**
- Overview
- Setup
  - Authentication Token
- Query
  - Output Description
- Related Links

## Overview

This notebook demonstrates how to use the Search API to find and browse multiple emission factors that match specific criteria. It allows users to:

- Search for emission factors using keywords (e.g., "gas", "electricity")
- Filter results by location and date
- Navigate through paginated results to explore all available options
- Compare different emission factors before selecting one for calculations

The Search API is valuable for exploration and discovery, helping users understand what emission factors are available before making calculations or when they need to compare multiple methodologies. This is particularly useful when exploring available emissions factors for a specific activity type or region. The results include detailed information about each factor, including its source, scope, unit of measurement, and emissions values, presented in a paginated format for easy navigation through large result sets.

## Setup


Ensure that you have Python installed in your system. Python 3+ is required.

<b>Note:</b> To run this notebook, you must first add your credentials to `config.read('../../../auth/secrets.ini')` in the following format:


```
[EAPI]
api.api_key = <Your GHG APIs API key>
api.client_id = <Your GHG APIs client Id>
api.org_id = <Your GHG APIs Org Id>

```

In [ ]:
# Install the packages below using pip/pip3 based on your python version.
%pip install pandas configparser IPython requests

In [1]:
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

### Authentication Token



Run the following code snippet to generate the Auth Bearer Token by using your api_key configured in secrets.ini.

In [2]:
config = configparser.RawConfigParser()
config.read(['../../../auth/secrets.ini','../../../auth/config.ini'])

EAPI_API_KEY        = config.get('EAPI', 'api.api_key')
EAPI_TENANT_ID      = config.get('EAPI', 'api.tenant_id')
EAPI_ORG_ID         = config.get('EAPI', 'api.org_id') 

EAPI_AUTH_ENDPOINT  = config.get('EAPI', 'api.auth_endpoint')
EAPI_BASE_URL       = config.get('EAPI', 'api.base_url')
EAPI_ENDPOINT       = f"{EAPI_BASE_URL}/factor/search"

EAPI_AUTH_CLIENT_ID = 'saascore-' + EAPI_TENANT_ID
EAPI_CLIENT_ID      = 'ghgemissions-' + EAPI_TENANT_ID

auth_request_headers: dict = {}
auth_request_headers["X-IBM-CLIENT-ID"] = EAPI_AUTH_CLIENT_ID
auth_request_headers["X-API-KEY"] = EAPI_API_KEY

verify = True

auth_url = f"{EAPI_AUTH_ENDPOINT}?orgId={EAPI_ORG_ID}"
              
response = requests.get(url = auth_url,
                        headers = auth_request_headers,
                        verify  = verify
                       )
if response.status_code == 200:
    jwt_token = response.text
    print("Authentication Success")
else:     
    print("Authentication Failed")
    print(response.text)

Authentication Success


## Query

The example request payload demonstrates how to search for emission factors using IBM Envizi - Emissions API by specifying location (India), a keyword ("gas"), pagination parameters (page 1, 30 results), and a reference date (January 4, 2024).

In [3]:
payload = {
  "location": {
    "country": "ind"
  },
  "activity": {
    "search": "gas"
  },
  "pagination": {
    "page": 1,
    "size": 30
  },
  "time": {
    "date": "2024-01-04"
  }
}

In [4]:
# Create the query headers
request_headers: dict = {}
request_headers["Content-Type"] = "application/json"
request_headers["x-ibm-client-id"] = EAPI_CLIENT_ID
request_headers["Authorization"] = "Bearer " + jwt_token

# Submit the request
response = requests.post(EAPI_ENDPOINT, 
                         headers = request_headers, 
                         data = json.dumps(payload))

For more information about allowable parameters for the payload, please see [Emissions API Developer Guide]().

In [5]:
if response.status_code == 200 and response.text:
    # Parse the JSON response
    response_json = response.json()
    
    # Extract the factors list
    factors_list = response_json.get('factors', [])
    
    # Create dataframe from factors list
    df_factors = pd.json_normalize(factors_list)
    
    # Display settings and results
    print(f"\nFound {len(factors_list)} matching factors\n")
    pd.set_option('display.max_colwidth', None)
    display_summary(df_factors)
else:
    print(f"Error: {response.status_code}")
    print(f"Response: {response.text}")


Found 30 matching factors



,factorSet,source,activityType,activityUnit,effectiveFrom,publishedFrom,region,factorId
0,Managed - eGRID & US Climate Leaders,"Supply Chain Greenhouse Gas Emission Factors v1.3 by NAICS-6, Publisher: U.S. EPA Office of Research and Development (ORD). Published on: 10-July-2024","Mining, except oil and gas",USD,01/01/2024,01/01/2024,Earth,162166
1,Managed - eGRID & US Climate Leaders,"Supply Chain Greenhouse Gas Emission Factors v1.3 by NAICS-6, Publisher: U.S. EPA Office of Research and Development (ORD). Published on: 10-July-2024","Mining, except oil and gas",USD,01/01/2024,01/01/2024,Earth,162167
2,Managed - eGRID & US Climate Leaders,"Supply Chain Greenhouse Gas Emission Factors v1.3 by NAICS-6, Publisher: U.S. EPA Office of Research and Development (ORD). Published on: 10-July-2024","Mining, except oil and gas",USD,01/01/2024,01/01/2024,Earth,162168
3,Managed - eGRID & US Climate Leaders,"Supply Chain Greenhouse Gas Emission Factors v1.3 by NAICS-6, Publisher: U.S. EPA Office of Research and Development (ORD). Published on: 10-July-2024","Mining, except oil and gas",USD,01/01/2024,01/01/2024,Earth,162169
4,Managed - eGRID & US Climate Leaders,"Supply Chain Greenhouse Gas Emission Factors v1.3 by NAICS-6, Publisher: U.S. EPA Office of Research and Development (ORD). Published on: 10-July-2024","Mining, except oil and gas",USD,01/01/2024,01/01/2024,Earth,162170
5,Managed - eGRID & US Climate Leaders,"Supply Chain Greenhouse Gas Emission Factors v1.3 by NAICS-6, Publisher: U.S. EPA Office of Research and Development (ORD). Published on: 10-July-2024","Mining, except oil and gas",USD,01/01/2024,01/01/2024,Earth,162171
6,Managed - eGRID & US Climate Leaders,US EPA. Feb 2024. EPA Centre for Climate Leadership. Emission Factors for Greenhouse Gas Inventories - https://www.epa.gov/climateleadership/ghg-emission-factors-hub,Natural Gas,100 cf,01/01/2024,01/01/2024,Earth,162923
7,Managed - eGRID & US Climate Leaders,US EPA. Feb 2024. EPA Centre for Climate Leadership. Emission Factors for Greenhouse Gas Inventories - https://www.epa.gov/climateleadership/ghg-emission-factors-hub,Natural Gas,1000 cf,01/01/2024,01/01/2024,Earth,162924
8,Managed - eGRID & US Climate Leaders,US EPA. Feb 2024. EPA Centre for Climate Leadership. Emission Factors for Greenhouse Gas Inventories - https://www.epa.gov/climateleadership/ghg-emission-factors-hub,Natural Gas,ccf,01/01/2024,01/01/2024,Earth,162925
9,Managed - eGRID & US Climate Leaders,US EPA. Feb 2024. EPA Centre for Climate Leadership. Emission Factors for Greenhouse Gas Inventories - https://www.epa.gov/climateleadership/ghg-emission-factors-hub,Natural Gas,cf,01/01/2024,01/01/2024,Earth,162926


### Output Description

<b>transactionId</b> - An Emissions API transaction id.

<b>totalCO2e</b>     - The total emissions as CO<sub>2</sub> equivalent (CO<sub>2</sub>e)

<b>CO2</b>           - The amount of CO<sub>2</sub> (Carbon Dioxide) in the CO<sub>2</sub>e value.

<b>CH4</b>           - The amount of CH<sub>4</sub> (Methane) in the CO<sub>2</sub>e value.

<b>N2O</b>           - The amount of N<sub>2</sub>O (Nitrous Oxide) in the CO<sub>2</sub>e value.

<b>HFC</b>           - The amount of HFCs (Hydrofluorocarbons) in the CO<sub>2</sub>e value.

<b>PFC</b>           - The amount of PFCs (Perfluorocarbons) in the CO<sub>2</sub>e value.

<b>SF6</b>           - The amount of SF<sub>6</sub> (Sulphur Hexafluoride) in the CO<sub>2</sub>e value.

<b>NF3</b>           - The amount of NF<sub>3</sub> (Nitrogen Trifluoride) in the CO<sub>2</sub>e value.

<b>bioCo2</b>        - The amount of bio CO<sub>2</sub> in the CO<sub>2</sub> value.

<b>indirectCo2e</b>  - The amount of CO<sub>2</sub>e that is indirect in the CO<sub>2</sub>e value.

<b>unit</b>          - The unit of measure of the values.

<b>description</b>   - A description of the source factor set of the factor used in the calculation.

## Related Links

[GHG Emissions API Developer Guide](https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/GHG+Emissions+API)